In [1]:
%reload_kedro
catalog = context.catalog

2021-08-24 11:56:33,364 - root - INFO - Registered hooks from 2 installed plugin(s): kedro-mlflow-0.7.3
2021-08-24 11:56:33,452 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2021-08-24 11:56:33,747 - root - INFO - ** Kedro project Spotify Recommendations
2021-08-24 11:56:33,749 - root - INFO - Defined global variable `context`, `session` and `catalog`
2021-08-24 11:56:33,758 - root - INFO - Registered line magic `run_viz`


In [3]:
import dask.dataframe as dd
import dask.distributed as d_dist
from dask.dataframe.core import repartition
from dask.distributed import Client

In [4]:
client = d_dist.client._get_global_client() or Client()

distributed.diskutils - INFO - Found stale lock file and directory '/Users/i854336/Documents/PERSONAL/spotify_playlists/spotify-recommender-training/notebooks/dask-worker-space/worker-u0u1rma1', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/i854336/Documents/PERSONAL/spotify_playlists/spotify-recommender-training/notebooks/dask-worker-space/worker-sdnjpj_d', purging


In [2]:
cleaned_mpd_playlists = catalog.load("cleaned_mpd_playlists")

2021-08-24 11:56:33,778 - kedro.io.data_catalog - INFO - Loading data from `cleaned_mpd_playlists` (ParquetDataSet)...


In [5]:
playlist_feature_cols = [
    "pid",
    "name",
    "collaborative",
    "modified_at",
    "num_followers",
    "num_edits"
]

In [6]:
# cleaned_mpd_playlists = cleaned_mpd_playlists.astype({"name": "string", "collaborative" : "bool"})

In [7]:
def str2bool(v): # Note the self to allow it to be in a class
    return v.lower() in ('yes', 'true', 't', '1', 'yea', 'verily')  # lower() is a method

In [8]:
def convert_text_bool_to_python_bool(val):
    return str2bool(val)

In [9]:
cleaned_mpd_playlists["collaborative"] = (
    cleaned_mpd_playlists["collaborative"]
    .map(convert_text_bool_to_python_bool)
)

In [10]:
from datetime import datetime

In [11]:
cleaned_mpd_playlists["modified_at"] = (
    cleaned_mpd_playlists["modified_at"]
    .map(datetime.utcfromtimestamp)
)

In [12]:
cleaned_mpd_playlists["modified_at"].head(3)

0   2017-04-29
1   2017-09-28
2   2017-09-18
Name: modified_at, dtype: datetime64[ns]

In [14]:
catalog.save("selected_mpd_playlist_features", cleaned_mpd_playlists)

2021-08-24 11:56:49,248 - kedro.io.data_catalog - INFO - Saving data to `selected_mpd_playlist_features` (ParquetDataSet)...
